## Evaluating single- and multi-task profile model in PyTorch with Katie's data loading

**Author:** Alex Tseng (amtseng@stanford.edu) -- Modified by Katie Spivakovsky

This notebook trains models on all permutations of SPI1 and GATA2 tasks as train, val, and test data. Pre-trained models' metrics can be loaded in if you don't want to spent 30 minutes re-training everything :) All of the data in the performance_metrics folder is already on the Google spreadsheet here: https://docs.google.com/spreadsheets/d/1BXC8yY809Zoo6NIPjypzYnx0Rv2m0Gq8Ni-azA7H0M4/edit?usp=sharing

TODO:
For thoroughness, I *should* include a "no reverse complement" option in statuses creation - but this is minor since usually we use rev-comp augmentation.

In [ ]:
# modify this for your own machine
%env CUDA_VISIBLE_DEVICES = 1
%config Completer.use_jedi = False

import pandas as pd
from all_functions import *

In [ ]:
# load in metrics from pre-trained models (rather than re-running all the training to get the metrics)
# modify this for your directories
single_task_metrics = pd.read_csv('/home/katie/bp_repo/multitask_profile_model_example/performance_metrics/single_task_metrics.csv')
multi_task_metrics = pd.read_csv('/home/katie/bp_repo/multitask_profile_model_example/performance_metrics/multi_task_metrics.csv')

In [ ]:
## single-task evaluation 
# you can run this (takes some time) or you can just look at the metrics that
# I already calculated (loaded in the previous cell, or found in the Google spreadsheet) 

# initialize metrics DataFrame
# if you change to fewer/more epochs, you'll need to update the row_names
row_names = ['Train task', 'Valid task', 'Test task',
            'Train epoch 1 loss', 'Valid epoch 1 loss','Train epoch 2 loss',
            'Valid epoch 2 loss','Train epoch 3 loss','Valid epoch 3 loss',
            'Train epoch 4 loss','Valid epoch 4 loss','Train epoch 5 loss',
            'Valid epoch 5 loss','Train epoch 6 loss','Valid epoch 6 loss',
            'Train epoch 7 loss','Valid epoch 7 loss','Train epoch 8 loss',
            'Valid epoch 8 loss','Train epoch 9 loss','Valid epoch 9 loss',
            'Train epoch 10 loss','Valid epoch 10 loss',
            'Test profile NLL','Test profile cross entropy','Test profile JSD',
            'Test profile Pearson','Test profile Spearman','Test profile MSE',
            'Test count Pearson','Test count Spearman','Test count MSE']

metrics = pd.DataFrame(data=row_names,columns=['Metrics'])

train_tasks = [ ['SPI1'], ['GATA2'] ]
val_tasks = [ ['SPI1'], ['GATA2'] ]
test_tasks = [ ['SPI1'], ['GATA2'] ]

num_permutations = len(train_tasks) * len(val_tasks) * len(test_tasks)

for train_task in train_tasks:
    for val_task in val_tasks:
        for test_task in test_tasks:
            combined_loss_and_metrics = evaluate(train_task, val_task, test_task, 1)
            metrics.insert( 1, str(num_permutations), combined_loss_and_metrics )
            num_permutations -= 1
            
metrics.set_index('Metrics',inplace=True)

# save as csv! to load it in, can use pd.read_csv(path)
# metrics.to_csv('/home/katie/bp_repo/multitask_profile_model_example/performance_metrics/single_task_metrics.csv')

In [ ]:
## multi-task evaluation
# you can run this (takes some time) or you can just look at the metrics that
# I already calculated (loaded in the previous cell, or found in the Google spreadsheet) 

# initialize metrics DataFrame
row_names = ['Train task', 'Valid task', 'Test task',
            'Train epoch 1 loss', 'Valid epoch 1 loss','Train epoch 2 loss',
            'Valid epoch 2 loss','Train epoch 3 loss','Valid epoch 3 loss',
            'Train epoch 4 loss','Valid epoch 4 loss','Train epoch 5 loss',
            'Valid epoch 5 loss','Train epoch 6 loss','Valid epoch 6 loss',
            'Train epoch 7 loss','Valid epoch 7 loss','Train epoch 8 loss',
            'Valid epoch 8 loss','Train epoch 9 loss','Valid epoch 9 loss',
            'Train epoch 10 loss','Valid epoch 10 loss',
            'Test profile NLL','Test profile cross entropy','Test profile JSD',
            'Test profile Pearson','Test profile Spearman','Test profile MSE',
            'Test count Pearson','Test count Spearman','Test count MSE']

metrics = pd.DataFrame(data=row_names,columns=['Metrics'])

# the "SPI1_copy" and "GATA2_copy" tasks are there for a control - you train a
# multi-task model but the data is redundant, so you should get metrics similar
# to the single-task version
train_tasks = [ ['SPI1', 'SPI1_copy'], ['GATA2', 'GATA2_copy'], ['SPI1','GATA2'] ]
val_tasks = [ ['SPI1', 'SPI1_copy'], ['GATA2', 'GATA2_copy'], ['SPI1','GATA2'] ]
test_tasks = [['SPI1', 'GATA2']]

num_permutations = len(train_tasks) * len(val_tasks) * len(test_tasks)

for train_task in train_tasks:
    for val_task in val_tasks:
        for test_task in test_tasks:
            combined_loss_and_metrics = evaluate(train_task, val_task, test_task, 2)
            metrics.insert( 1, str(num_permutations), combined_loss_and_metrics )
            num_permutations -= 1
            
metrics.set_index('Metrics',inplace=True)

# save as csv! to load it in, can use pd.read_csv(path)
# metrics.to_csv('/home/katie/bp_repo/multitask_profile_model_example/performance_metrics/multi_task_metrics.csv')